In [24]:
using Pkg
Pkg.activate(dirname(dirname(@__DIR__)))
using Macro
using Gurobi
using Plots
using DataFrames, CSV
using BenchmarkTools

  Activating project at `~/Macro`


In [2]:
using JuMP

In [5]:
case_path = @__DIR__
println("###### ###### ######")
println("Running case at $(case_path)")

system = Macro.load_system(case_path)

model = Macro.generate_model(system)

Macro.set_optimizer(model, Gurobi.Optimizer);

Macro.set_optimizer_attributes(model, "BarConvTol"=>1e-3,"Crossover" => 0, "Method" => 2)

Macro.optimize!(model)

###### ###### ######
Running case at /home/al3792/Macro/ExampleSystems/three_zones_macro_genx_cement


┌ Info: Loading JSON data from /home/al3792/Macro/ExampleSystems/three_zones_macro_genx_cement/system_data.json
└ @ Macro /home/al3792/Macro/src/load_inputs/load_macroobject.jl:135
┌ Info: Loading JSON data from /home/al3792/Macro/ExampleSystems/three_zones_macro_genx_cement/system_data.json
└ @ Macro /home/al3792/Macro/src/load_inputs/load_macroobject.jl:135
┌ Info: Loading JSON data from system/nodes.json
└ @ Macro /home/al3792/Macro/src/load_inputs/load_macroobject.jl:135
┌ Info: Loading CSV data from system/demand.csv
└ @ Macro /home/al3792/Macro/src/load_inputs/file_io/csv.jl:12
┌ Info: Loading CSV data from system/demand.csv
└ @ Macro /home/al3792/Macro/src/load_inputs/file_io/csv.jl:14
┌ Info: Loading CSV data from system/hourly_cement_demand.csv
└ @ Macro /home/al3792/Macro/src/load_inputs/file_io/csv.jl:12
┌ Info: Loading CSV data from system/hourly_cement_demand.csv
└ @ Macro /home/al3792/Macro/src/load_inputs/file_io/csv.jl:14
┌ Info: Loading JSON data from assets/cementplan

system.settings.Scaling = true


┌ Info: Scaling system data to GWh | ktons | M$
└ @ Macro /home/al3792/Macro/src/model/scaling.jl:21
┌ Info: Starting model generation
└ @ Macro /home/al3792/Macro/src/generate_model.jl:3
┌ Info: Adding linking variables
└ @ Macro /home/al3792/Macro/src/generate_model.jl:17
┌ Info: Defining available capacity
└ @ Macro /home/al3792/Macro/src/generate_model.jl:20
┌ Info: Generating planning model
└ @ Macro /home/al3792/Macro/src/generate_model.jl:23
┌ Info: Generating operational model
└ @ Macro /home/al3792/Macro/src/generate_model.jl:26


Set parameter LicenseID to value 197246
Set parameter GURO_PAR_SPECIAL
Set parameter TokenServer to value "license.rc.princeton.edu"
Set parameter BarConvTol to value 0.001
Set parameter Crossover to value 0
Set parameter Method to value 2
Set parameter Method to value 2
Set parameter Crossover to value 0
Set parameter BarConvTol to value 0.001
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (linux64 - "Red Hat Enterprise Linux 8.10 (Ootpa)")

CPU model: Intel(R) Xeon(R) Gold 6246R CPU @ 3.40GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 32 physical cores, 32 logical processors, using up to 32 threads

Optimize a model with 998666 rows, 683329 columns and 3292716 nonzeros
Model fingerprint: 0x20384686
Coefficient statistics:
  Matrix range     [4e-10, 2e+01]
  Objective range  [2e-05, 5e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e-03, 2e+01]
Presolve removed 377242 rows and 272121 columns
Presolve time: 2.24s
Presolved: 621424 rows, 411208 columns, 2205912 

┌ Info: Model generation complete, it took 23.83653998374939 seconds
└ @ Macro /home/al3792/Macro/src/generate_model.jl:31


In [20]:
case = "capacity_no_co2_cap"
capacity_results = Macro.get_optimal_asset_capacity(system)
results_dir = joinpath(case_path, "results")
mkpath(results_dir)
Macro.write_csv(joinpath(results_dir, case * "_capacity.csv"), capacity_results)
println("wrote results")

wrote results


# Cement Constraints

In [24]:
system.assets[3].co2_edge.constraints[1]

CapacityConstraint(missing, missing, 1-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape},1,...} with index sets:
    Dimension 1, 1:1:8760
And data, a 8760-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape}}:
 -0.0009000000000000001 vNEWCAP_sorbent_dac_MA_co2_edge + 0.0009000000000000001 vRETCAP_sorbent_dac_MA_co2_edge + vFLOW_sorbent_dac_MA_co2_edge[1] ≤ 0
 -0.0009000000000000001 vNEWCAP_sorbent_dac_MA_co2_edge + 0.0009000000000000001 vRETCAP_sorbent_dac_MA_co2_edge + vFLOW_sorbent_dac_MA_co2_edge[2] ≤ 0
 -0.0009000000000000001 vNEWCAP_sorbent_dac_MA_co2_edge + 0.0009000000000000001 vRETCAP_sorbent_dac_MA_co2_edge + vFLOW_sorbent_dac_MA_co2_edge[3] ≤ 0
 -0.0009000000000000001 vNEWCAP_sorbent_dac_MA_co2_edge + 0.0009000000000000001 vRETCAP_sorbe

In [25]:
system.assets[3].co2_edge.constraints[2]

RampingLimitConstraint(missing, missing, 2-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape},2,...} with index sets:
    Dimension 1, [:RampUp, :RampDown]
    Dimension 2, 1:1:8760
And data, a 2×8760 Matrix{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape}}:
 -0.001 vNEWCAP_sorbent_dac_MA_co2_edge + 0.001 vRETCAP_sorbent_dac_MA_co2_edge + vFLOW_sorbent_dac_MA_co2_edge[1] - vFLOW_sorbent_dac_MA_co2_edge[8760] ≤ 0  …  -0.001 vNEWCAP_sorbent_dac_MA_co2_edge + 0.001 vRETCAP_sorbent_dac_MA_co2_edge - vFLOW_sorbent_dac_MA_co2_edge[8759] + vFLOW_sorbent_dac_MA_co2_edge[8760] ≤ 0
 -0.001 vNEWCAP_sorbent_dac_MA_co2_edge + 0.001 vRETCAP_sorbent_dac_MA_co2_edge - vFLOW_sorbent_dac_MA_co2_edge[1] + vFLOW_sorbent_dac_MA_co2_edge[8760] ≤ 0     -0.001 vNEWCAP_sorbent_dac_MA

In [ ]:
system.assets[1].cement_materials_edge.constraints

AbstractTypeConstraint[]

In [32]:
system.assets[1].elec_edge.constraints

AbstractTypeConstraint[]

In [33]:
system.assets[1].fuel_edge.constraints

AbstractTypeConstraint[]

In [34]:
system.assets[1].cement_edge.constraints

1-element Vector{AbstractTypeConstraint}:
 CapacityConstraint(missing, missing, 1-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape},1,...} with index sets:
    Dimension 1, 1:1:8760
And data, a 8760-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape}}:
 -0.001 vNEWCAP_cement_MA_cement_edge + 0.001 vRETCAP_cement_MA_cement_edge + vFLOW_cement_MA_cement_edge[1] ≤ 0
 -0.001 vNEWCAP_cement_MA_cement_edge + 0.001 vRETCAP_cement_MA_cement_edge + vFLOW_cement_MA_cement_edge[2] ≤ 0
 -0.001 vNEWCAP_cement_MA_cement_edge + 0.001 vRETCAP_cement_MA_cement_edge + vFLOW_cement_MA_cement_edge[3] ≤ 0
 -0.001 vNEWCAP_cement_MA_cement_edge + 0.001 vRETCAP_cement_MA_cement_edge + vFLOW_cement_MA_cement_edge[4] ≤ 0
 -0.001 vNEWCAP_cement_MA_cement_edge + 0.001 vRETC

In [35]:
system.assets[1].co2_edge.constraints

AbstractTypeConstraint[]

# Power Line Constraints

In [37]:
system.assets[4].elec_edge.constraints[1]

CapacityConstraint(missing, missing, 2-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape},2,...} with index sets:
    Dimension 1, [-1, 1]
    Dimension 2, 1:1:8760
And data, a 2×8760 Matrix{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape}}:
 -0.001 vNEWCAP_MA_to_CT_elec_edge + 0.001 vRETCAP_MA_to_CT_elec_edge - vFLOW_MA_to_CT_elec_edge[1] ≤ 2.95  …  -0.001 vNEWCAP_MA_to_CT_elec_edge + 0.001 vRETCAP_MA_to_CT_elec_edge - vFLOW_MA_to_CT_elec_edge[8760] ≤ 2.95
 -0.001 vNEWCAP_MA_to_CT_elec_edge + 0.001 vRETCAP_MA_to_CT_elec_edge + vFLOW_MA_to_CT_elec_edge[1] ≤ 2.95     -0.001 vNEWCAP_MA_to_CT_elec_edge + 0.001 vRETCAP_MA_to_CT_elec_edge + vFLOW_MA_to_CT_elec_edge[8760] ≤ 2.95)

In [38]:
system.assets[4].elec_edge.constraints[2]

MaxCapacityConstraint(missing, missing, 0.001 vNEWCAP_MA_to_CT_elec_edge - 0.001 vRETCAP_MA_to_CT_elec_edge ≤ 2.95)

# Storage Constraints

In [41]:
system.assets[6].battery_transform.constraints[1]

StorageCapacityConstraint(missing, missing, 1-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape},1,...} with index sets:
    Dimension 1, 1:1:8760
And data, a 8760-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape}}:
 -vNEWCAPSTOR_storage_MA_storage + vRETCAPSTOR_storage_MA_storage + vSTOR_storage_MA_storage[1] ≤ 0
 -vNEWCAPSTOR_storage_MA_storage + vRETCAPSTOR_storage_MA_storage + vSTOR_storage_MA_storage[2] ≤ 0
 -vNEWCAPSTOR_storage_MA_storage + vRETCAPSTOR_storage_MA_storage + vSTOR_storage_MA_storage[3] ≤ 0
 -vNEWCAPSTOR_storage_MA_storage + vRETCAPSTOR_storage_MA_storage + vSTOR_storage_MA_storage[4] ≤ 0
 -vNEWCAPSTOR_storage_MA_storage + vRETCAPSTOR_storage_MA_storage + vSTOR_storage_MA_storage[5] ≤ 0
 -vNEWCAPSTOR_storage_MA_storage + vRET

In [42]:
system.assets[6].battery_transform.constraints[2]

StorageSymmetricCapacityConstraint(missing, missing, 1-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape},1,...} with index sets:
    Dimension 1, 1:1:8760
And data, a 8760-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape}}:
 -0.001 vNEWCAP_storage_MA_discharge_edge + 0.001 vRETCAP_storage_MA_discharge_edge + vFLOW_storage_MA_discharge_edge[1] + vFLOW_storage_MA_charge_edge[1] ≤ 0
 -0.001 vNEWCAP_storage_MA_discharge_edge + 0.001 vRETCAP_storage_MA_discharge_edge + vFLOW_storage_MA_discharge_edge[2] + vFLOW_storage_MA_charge_edge[2] ≤ 0
 -0.001 vNEWCAP_storage_MA_discharge_edge + 0.001 vRETCAP_storage_MA_discharge_edge + vFLOW_storage_MA_discharge_edge[3] + vFLOW_storage_MA_charge_edge[3] ≤ 0
 -0.001 vNEWCAP_storage_MA_discharge_edge + 0.001 vRE

In [43]:
system.assets[6].battery_transform.constraints[3]

BalanceConstraint(missing, missing, 2-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape},2,...} with index sets:
    Dimension 1, [:storage]
    Dimension 2, 1:1:8760
And data, a 1×8760 Matrix{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape}}:
 -vSTOR_storage_MA_storage[1] + vSTOR_storage_MA_storage[8760] - 1.0869565217391304 vFLOW_storage_MA_discharge_edge[1] + 0.92 vFLOW_storage_MA_charge_edge[1] = 0  …  vSTOR_storage_MA_storage[8759] - vSTOR_storage_MA_storage[8760] - 1.0869565217391304 vFLOW_storage_MA_discharge_edge[8760] + 0.92 vFLOW_storage_MA_charge_edge[8760] = 0)

In [44]:
system.assets[6].discharge_edge.constraints

1-element Vector{AbstractTypeConstraint}:
 CapacityConstraint(missing, missing, 1-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape},1,...} with index sets:
    Dimension 1, 1:1:8760
And data, a 8760-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape}}:
 -0.001 vNEWCAP_storage_MA_discharge_edge + 0.001 vRETCAP_storage_MA_discharge_edge + vFLOW_storage_MA_discharge_edge[1] ≤ 0
 -0.001 vNEWCAP_storage_MA_discharge_edge + 0.001 vRETCAP_storage_MA_discharge_edge + vFLOW_storage_MA_discharge_edge[2] ≤ 0
 -0.001 vNEWCAP_storage_MA_discharge_edge + 0.001 vRETCAP_storage_MA_discharge_edge + vFLOW_storage_MA_discharge_edge[3] ≤ 0
 -0.001 vNEWCAP_storage_MA_discharge_edge + 0.001 vRETCAP_storage_MA_discharge_edge + vFLOW_storage_MA_discharge_edge[4] ≤ 0
 -0

In [5]:
system.assets[6].charge_edge.constraints

AbstractTypeConstraint[]

# Natural Gas

In [7]:
system.assets[11].thermal_transform.constraints

1-element Vector{AbstractTypeConstraint}:
 BalanceConstraint(missing, missing, 2-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape},2,...} with index sets:
    Dimension 1, [:emissions, :energy]
    Dimension 2, 1:1:8760
And data, a 2×8760 Matrix{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape}}:
 0.181048235160161 vFLOW_ng_ME_fuel_edge[1] - vFLOW_ng_ME_co2_edge[1] = 0                          …  0.181048235160161 vFLOW_ng_ME_fuel_edge[8760] - vFLOW_ng_ME_co2_edge[8760] = 0
 -vFLOW_ng_ME_elec_edge[1] - 0.146535535 vSTART_ng_ME_elec_edge[1] + vFLOW_ng_ME_fuel_edge[1] = 0     -vFLOW_ng_ME_elec_edge[8760] - 0.146535535 vSTART_ng_ME_elec_edge[8760] + vFLOW_ng_ME_fuel_edge[8760] = 0)

In [9]:
system.assets[11].elec_edge.constraints[1]

CapacityConstraint(missing, missing, 1-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape},1,...} with index sets:
    Dimension 1, 1:1:8760
And data, a 8760-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape}}:
 vFLOW_ng_ME_elec_edge[1] - 0.25 vCOMMIT_ng_ME_elec_edge[1] ≤ 0
 vFLOW_ng_ME_elec_edge[2] - 0.25 vCOMMIT_ng_ME_elec_edge[2] ≤ 0
 vFLOW_ng_ME_elec_edge[3] - 0.25 vCOMMIT_ng_ME_elec_edge[3] ≤ 0
 vFLOW_ng_ME_elec_edge[4] - 0.25 vCOMMIT_ng_ME_elec_edge[4] ≤ 0
 vFLOW_ng_ME_elec_edge[5] - 0.25 vCOMMIT_ng_ME_elec_edge[5] ≤ 0
 vFLOW_ng_ME_elec_edge[6] - 0.25 vCOMMIT_ng_ME_elec_edge[6] ≤ 0
 vFLOW_ng_ME_elec_edge[7] - 0.25 vCOMMIT_ng_ME_elec_edge[7] ≤ 0
 vFLOW_ng_ME_elec_edge[8] - 0.25 vCOMMIT_ng_ME_elec_edge[8] ≤ 0
 vFLOW_ng_ME_elec_edge[9] - 0.25 v

In [5]:
system.assets[11].elec_edge.constraints[2]

RampingLimitConstraint(missing, missing, 2-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape},2,...} with index sets:
    Dimension 1, [:RampUp, :RampDown]
    Dimension 2, 1:1:8760
And data, a 2×8760 Matrix{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape}}:
 vFLOW_ng_ME_elec_edge[1] - vFLOW_ng_ME_elec_edge[8760] - 0.16 vCOMMIT_ng_ME_elec_edge[1] + 0.1185 vSHUT_ng_ME_elec_edge[1] ≤ 0                                                 …  -vFLOW_ng_ME_elec_edge[8759] + vFLOW_ng_ME_elec_edge[8760] - 0.16 vCOMMIT_ng_ME_elec_edge[8760] + 0.1185 vSHUT_ng_ME_elec_edge[8760] ≤ 0
 -vFLOW_ng_ME_elec_edge[1] + vFLOW_ng_ME_elec_edge[8760] - 0.16 vCOMMIT_ng_ME_elec_edge[1] + 0.27849999999999997 vSTART_ng_ME_elec_edge[1] - 0.16 vSHUT_ng_ME_elec_edge[1] ≤ 0     vFLOW_ng_ME_ele

In [7]:
system.assets[10].elec_edge.constraints[2]

RampingLimitConstraint(missing, missing, 2-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape},2,...} with index sets:
    Dimension 1, [:RampUp, :RampDown]
    Dimension 2, 1:1:8760
And data, a 2×8760 Matrix{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape}}:
 vFLOW_ng_CT_elec_edge[1] - vFLOW_ng_CT_elec_edge[8760] - 0.16 vCOMMIT_ng_CT_elec_edge[1] + 0.0845 vSHUT_ng_CT_elec_edge[1] ≤ 0                                    …  -vFLOW_ng_CT_elec_edge[8759] + vFLOW_ng_CT_elec_edge[8760] - 0.16 vCOMMIT_ng_CT_elec_edge[8760] + 0.0845 vSHUT_ng_CT_elec_edge[8760] ≤ 0
 -vFLOW_ng_CT_elec_edge[1] + vFLOW_ng_CT_elec_edge[8760] - 0.16 vCOMMIT_ng_CT_elec_edge[1] + 0.2445 vSTART_ng_CT_elec_edge[1] - 0.16 vSHUT_ng_CT_elec_edge[1] ≤ 0     vFLOW_ng_CT_elec_edge[8759] - vFLOW_ng_CT

In [8]:
system.assets[9].elec_edge.constraints[2]

RampingLimitConstraint(missing, missing, 2-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape},2,...} with index sets:
    Dimension 1, [:RampUp, :RampDown]
    Dimension 2, 1:1:8760
And data, a 2×8760 Matrix{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape}}:
 vFLOW_ng_MA_elec_edge[1] - vFLOW_ng_MA_elec_edge[8760] - 0.16 vCOMMIT_ng_MA_elec_edge[1] + 0.117 vSHUT_ng_MA_elec_edge[1] ≤ 0                                    …  -vFLOW_ng_MA_elec_edge[8759] + vFLOW_ng_MA_elec_edge[8760] - 0.16 vCOMMIT_ng_MA_elec_edge[8760] + 0.117 vSHUT_ng_MA_elec_edge[8760] ≤ 0
 -vFLOW_ng_MA_elec_edge[1] + vFLOW_ng_MA_elec_edge[8760] - 0.16 vCOMMIT_ng_MA_elec_edge[1] + 0.277 vSTART_ng_MA_elec_edge[1] - 0.16 vSHUT_ng_MA_elec_edge[1] ≤ 0     vFLOW_ng_MA_elec_edge[8759] - vFLOW_ng_MA_el

In [11]:
system.assets[11].elec_edge.constraints[3]

MinFlowConstraint(missing, missing, 1-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.GreaterThan{Float64}}, ScalarShape},1,...} with index sets:
    Dimension 1, 1:1:8760
And data, a 8760-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.GreaterThan{Float64}}, ScalarShape}}:
 vFLOW_ng_ME_elec_edge[1] - 0.1185 vCOMMIT_ng_ME_elec_edge[1] ≥ 0
 vFLOW_ng_ME_elec_edge[2] - 0.1185 vCOMMIT_ng_ME_elec_edge[2] ≥ 0
 vFLOW_ng_ME_elec_edge[3] - 0.1185 vCOMMIT_ng_ME_elec_edge[3] ≥ 0
 vFLOW_ng_ME_elec_edge[4] - 0.1185 vCOMMIT_ng_ME_elec_edge[4] ≥ 0
 vFLOW_ng_ME_elec_edge[5] - 0.1185 vCOMMIT_ng_ME_elec_edge[5] ≥ 0
 vFLOW_ng_ME_elec_edge[6] - 0.1185 vCOMMIT_ng_ME_elec_edge[6] ≥ 0
 vFLOW_ng_ME_elec_edge[7] - 0.1185 vCOMMIT_ng_ME_elec_edge[7] ≥ 0
 vFLOW_ng_ME_elec_edge[8] - 0.1185 vCOMMIT_ng_ME_elec_edge[8] ≥ 0
 vFLOW_ng_ME_

In [12]:
system.assets[11].elec_edge.constraints[4]

MinUpTimeConstraint(missing, missing, 1-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.GreaterThan{Float64}}, ScalarShape},1,...} with index sets:
    Dimension 1, 1:1:8760
And data, a 8760-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.GreaterThan{Float64}}, ScalarShape}}:
 vCOMMIT_ng_ME_elec_edge[1] - vSTART_ng_ME_elec_edge[1] - vSTART_ng_ME_elec_edge[8756] - vSTART_ng_ME_elec_edge[8757] - vSTART_ng_ME_elec_edge[8758] - vSTART_ng_ME_elec_edge[8759] - vSTART_ng_ME_elec_edge[8760] ≥ 0
 vCOMMIT_ng_ME_elec_edge[2] - vSTART_ng_ME_elec_edge[1] - vSTART_ng_ME_elec_edge[2] - vSTART_ng_ME_elec_edge[8757] - vSTART_ng_ME_elec_edge[8758] - vSTART_ng_ME_elec_edge[8759] - vSTART_ng_ME_elec_edge[8760] ≥ 0
 vCOMMIT_ng_ME_elec_edge[3] - vSTART_ng_ME_elec_edge[1] - vSTART_ng_ME_elec_edge[2] - vSTART_ng_ME_elec_edge[3] 

In [13]:
system.assets[11].elec_edge.constraints[5]

MinDownTimeConstraint(missing, missing, 1-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.GreaterThan{Float64}}, ScalarShape},1,...} with index sets:
    Dimension 1, 1:1:8760
And data, a 8760-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.GreaterThan{Float64}}, ScalarShape}}:
 vNEWCAP_ng_ME_elec_edge - vRETCAP_ng_ME_elec_edge - vCOMMIT_ng_ME_elec_edge[1] - vSHUT_ng_ME_elec_edge[1] - vSHUT_ng_ME_elec_edge[8756] - vSHUT_ng_ME_elec_edge[8757] - vSHUT_ng_ME_elec_edge[8758] - vSHUT_ng_ME_elec_edge[8759] - vSHUT_ng_ME_elec_edge[8760] ≥ 0
 vNEWCAP_ng_ME_elec_edge - vRETCAP_ng_ME_elec_edge - vCOMMIT_ng_ME_elec_edge[2] - vSHUT_ng_ME_elec_edge[1] - vSHUT_ng_ME_elec_edge[2] - vSHUT_ng_ME_elec_edge[8757] - vSHUT_ng_ME_elec_edge[8758] - vSHUT_ng_ME_elec_edge[8759] - vSHUT_ng_ME_elec_edge[8760] ≥ 0
 vNEWCAP_ng_ME_ele

In [5]:
system.assets[11].fuel_edge.constraints

AbstractTypeConstraint[]

In [ ]:
system.assets[11].co2_edge.constraints

AbstractTypeConstraint[]

# VRE Constraints

In [15]:
system.assets[14].energy_transform.constraints

AbstractTypeConstraint[]

In [8]:
system.assets[14].edge.constraints

1-element Vector{AbstractTypeConstraint}:
 CapacityConstraint(missing, missing, 1-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape},1,...} with index sets:
    Dimension 1, 1:1:8760
And data, a 8760-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape}}:
 -0.000569944978 vNEWCAP_onshore_wind_CT_edge + 0.000569944978 vRETCAP_onshore_wind_CT_edge + vFLOW_onshore_wind_CT_edge[1] ≤ 0
 -0.000623258889 vNEWCAP_onshore_wind_CT_edge + 0.000623258889 vRETCAP_onshore_wind_CT_edge + vFLOW_onshore_wind_CT_edge[2] ≤ 0
 -0.0006941884160000001 vNEWCAP_onshore_wind_CT_edge + 0.0006941884160000001 vRETCAP_onshore_wind_CT_edge + vFLOW_onshore_wind_CT_edge[3] ≤ 0
 -0.0006473996640000001 vNEWCAP_onshore_wind_CT_edge + 0.0006473996640000001 vRETCAP_onshore_wind_CT_edge

# Location constraints

In [4]:
system.locations

8-element Vector{Node}:
 Node{NaturalGas}(:ng_source, Macro.TimeData{NaturalGas}(1:1:8760, 1, StepRange{Int64, Int64}[1:1:8760], [1], Dict(1 => 1.0)), Dict(:exogenous => Dict()), AbstractTypeConstraint[], Dict{Any, Any}(:exogenous => 1-dimensional DenseAxisArray{AffExpr,1,...} with index sets:
    Dimension 1, 1:1:8760
And data, a 8760-element Vector{AffExpr}:
 -vFLOW_cement_MA_fuel_edge[1] - vFLOW_cement_CT_fuel_edge[1] - vFLOW_cement_ME_fuel_edge[1] - vFLOW_ng_MA_fuel_edge[1] - vFLOW_ng_CT_fuel_edge[1] - vFLOW_ng_ME_fuel_edge[1]
 -vFLOW_cement_MA_fuel_edge[2] - vFLOW_cement_CT_fuel_edge[2] - vFLOW_cement_ME_fuel_edge[2] - vFLOW_ng_MA_fuel_edge[2] - vFLOW_ng_CT_fuel_edge[2] - vFLOW_ng_ME_fuel_edge[2]
 -vFLOW_cement_MA_fuel_edge[3] - vFLOW_cement_CT_fuel_edge[3] - vFLOW_cement_ME_fuel_edge[3] - vFLOW_ng_MA_fuel_edge[3] - vFLOW_ng_CT_fuel_edge[3] - vFLOW_ng_ME_fuel_edge[3]
 -vFLOW_cement_MA_fuel_edge[4] - vFLOW_cement_CT_fuel_edge[4] - vFLOW_cement_ME_fuel_edge[4] - vFLOW_ng_MA_fuel_edg

In [ ]:
# Emissions at a node
system.locations[2].balance_data[:emissions]

In [ ]:
# Electricity constraints
system.locations[2].constraints[1]

BalanceConstraint(missing, missing, 2-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape},2,...} with index sets:
    Dimension 1, [:demand]
    Dimension 2, 1:1:8760
And data, a 1×8760 Matrix{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape}}:
 -7.85 vREF + vNSD_elec_MA[1,1] + vNSD_elec_MA[2,1] + vNSD_elec_MA[3,1] + vNSD_elec_MA[4,1] - vFLOW_cement_MA_elec_edge[1] - vFLOW_MA_to_CT_elec_edge[1] - vFLOW_MA_to_ME_elec_edge[1] + vFLOW_storage_MA_discharge_edge[1] - vFLOW_storage_MA_charge_edge[1] + vFLOW_ng_MA_elec_edge[1] + vFLOW_solar_pv_MA_edge[1] = 0  …  -8.089 vREF + vNSD_elec_MA[1,8760] + vNSD_elec_MA[2,8760] + vNSD_elec_MA[3,8760] + vNSD_elec_MA[4,8760] - vFLOW_cement_MA_elec_edge[8760] - vFLOW_MA_to_CT_elec_edge[8760] - vFLOW_MA_to_ME_elec_edge[8760] + vFLOW

In [34]:
system.locations[2].constraints[2]

MaxNonServedDemandConstraint(missing, missing, 1-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape},1,...} with index sets:
    Dimension 1, 1:1:8760
And data, a 8760-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape}}:
 vNSD_elec_MA[1,1] + vNSD_elec_MA[2,1] + vNSD_elec_MA[3,1] + vNSD_elec_MA[4,1] ≤ 7.85
 vNSD_elec_MA[1,2] + vNSD_elec_MA[2,2] + vNSD_elec_MA[3,2] + vNSD_elec_MA[4,2] ≤ 7.424
 vNSD_elec_MA[1,3] + vNSD_elec_MA[2,3] + vNSD_elec_MA[3,3] + vNSD_elec_MA[4,3] ≤ 7.107
 vNSD_elec_MA[1,4] + vNSD_elec_MA[2,4] + vNSD_elec_MA[3,4] + vNSD_elec_MA[4,4] ≤ 6.947
 vNSD_elec_MA[1,5] + vNSD_elec_MA[2,5] + vNSD_elec_MA[3,5] + vNSD_elec_MA[4,5] ≤ 6.922
 vNSD_elec_MA[1,6] + vNSD_elec_MA[2,6] + vNSD_elec_MA[3,6] + vNSD_elec_MA[4,6] ≤ 7.045
 vNSD_elec_MA[1

In [35]:
system.locations[2].constraints[3]

MaxNonServedDemandPerSegmentConstraint(missing, missing, 2-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape},2,...} with index sets:
    Dimension 1, 1:4
    Dimension 2, 1:1:8760
And data, a 4×8760 Matrix{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape}}:
 vNSD_elec_MA[1,1] ≤ 7.85                  …  vNSD_elec_MA[1,8760] ≤ 8.089
 vNSD_elec_MA[2,1] ≤ 0.314                    vNSD_elec_MA[2,8760] ≤ 0.32356
 vNSD_elec_MA[3,1] ≤ 0.18839999999999998      vNSD_elec_MA[3,8760] ≤ 0.194136
 vNSD_elec_MA[4,1] ≤ 0.023549999999999998     vNSD_elec_MA[4,8760] ≤ 0.024267)

In [ ]:
# Cement Demand
system.locations[6].constraints

1-element Vector{AbstractTypeConstraint}:
 BalanceConstraint(missing, missing, 2-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape},2,...} with index sets:
    Dimension 1, [:demand]
    Dimension 2, 1:1:8760
And data, a 1×8760 Matrix{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape}}:
 -0.001 vREF + vFLOW_cement_MA_cement_edge[1] = 0  …  -0.001 vREF + vFLOW_cement_MA_cement_edge[8760] = 0)

# Scratch

In [5]:
for con in JuMP.all_constraints(model, include_variable_in_set_constraints=false)
    println(con)
end

vCOMMIT_ng_MA_elec_edge[1] - vCOMMIT_ng_MA_elec_edge[8760] - vSTART_ng_MA_elec_edge[1] + vSHUT_ng_MA_elec_edge[1] = 0
-vCOMMIT_ng_MA_elec_edge[1] + vCOMMIT_ng_MA_elec_edge[2] - vSTART_ng_MA_elec_edge[2] + vSHUT_ng_MA_elec_edge[2] = 0
-vCOMMIT_ng_MA_elec_edge[2] + vCOMMIT_ng_MA_elec_edge[3] - vSTART_ng_MA_elec_edge[3] + vSHUT_ng_MA_elec_edge[3] = 0
-vCOMMIT_ng_MA_elec_edge[3] + vCOMMIT_ng_MA_elec_edge[4] - vSTART_ng_MA_elec_edge[4] + vSHUT_ng_MA_elec_edge[4] = 0
-vCOMMIT_ng_MA_elec_edge[4] + vCOMMIT_ng_MA_elec_edge[5] - vSTART_ng_MA_elec_edge[5] + vSHUT_ng_MA_elec_edge[5] = 0
-vCOMMIT_ng_MA_elec_edge[5] + vCOMMIT_ng_MA_elec_edge[6] - vSTART_ng_MA_elec_edge[6] + vSHUT_ng_MA_elec_edge[6] = 0
-vCOMMIT_ng_MA_elec_edge[6] + vCOMMIT_ng_MA_elec_edge[7] - vSTART_ng_MA_elec_edge[7] + vSHUT_ng_MA_elec_edge[7] = 0
-vCOMMIT_ng_MA_elec_edge[7] + vCOMMIT_ng_MA_elec_edge[8] - vSTART_ng_MA_elec_edge[8] + vSHUT_ng_MA_elec_edge[8] = 0
-vCOMMIT_ng_MA_elec_edge[8] + vCOMMIT_ng_MA_elec_edge[9] - vSTART_ng_M

Excessive output truncated after 524350 bytes.

-vCOMMIT_ng_MA_elec_edge[4040] + vCOMMIT_ng_MA_elec_edge[4041] - vSTART_ng_MA_elec_edge[4041] + vSHUT_ng_MA_elec_edge[4041] = 0
-vCOMMIT_ng_MA_elec_edge[4041] + vCOMMIT_ng_MA_elec_edge[4042] - vSTART_ng_MA_elec_edge[4042] + vSHUT_ng_MA_elec_edge[4042] = 0
-vCOMMIT_ng_MA_elec_edge[4042] + vCOMMIT_ng_MA_elec_edge[4043] - vSTART_ng_MA_elec_edge[4043] + vSHUT_ng_MA_elec_edge[4043] = 0
-vCOMMIT_ng_MA_elec_edge[4043] + vCOMMIT_ng_MA_elec_edge[4044] - vSTART_ng_MA_elec_edge[4044] + vSHUT_ng_MA_elec_edge[4044] = 0
-vCOMMIT_ng_MA_elec_edge[4044] + vCOMMIT_ng_MA_elec_edge[4045] - vSTART_ng_MA_elec_edge[4045] + vSHUT_ng_MA_elec_edge[4045] = 0
-vCOMMIT_ng_MA_elec_edge[4045] + vCOMMIT_ng_MA_elec_edge[4046] - vSTART_ng_MA_elec_edge[4046] + vSHUT_ng_MA_elec_edge[4046] = 0
-vCOMMIT_ng_MA_elec_edge[4046] + vCOMMIT_ng_MA_elec_edge[4047] - vSTART_ng_MA_elec_edge[4047] + vSHUT_ng_MA_elec_edge[4047] = 0
-vCOMMIT_ng_MA_elec_edge[4047] + vCOMMIT_ng_MA_elec_edge[4048] - vSTART_ng_MA_elec_edge[4048] + vSHUT_ng

In [ ]:
system.locations[4].constraints

3-element Vector{AbstractTypeConstraint}:
 BalanceConstraint(missing, missing, 2-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape},2,...} with index sets:
    Dimension 1, [:demand]
    Dimension 2, 1:1:8760
And data, a 1×8760 Matrix{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape}}:
 -1.07 vREF + vNSD_elec_ME[1,1] + vNSD_elec_ME[2,1] + vNSD_elec_ME[3,1] + vNSD_elec_ME[4,1] - vFLOW_cement_ME_elec_edge[1] + vFLOW_MA_to_ME_elec_edge[1] + vFLOW_storage_ME_discharge_edge[1] - vFLOW_storage_ME_charge_edge[1] + vFLOW_ng_ME_elec_edge[1] + vFLOW_onshore_wind_ME_edge[1] = 0  …  -1.092 vREF + vNSD_elec_ME[1,8760] + vNSD_elec_ME[2,8760] + vNSD_elec_ME[3,8760] + vNSD_elec_ME[4,8760] - vFLOW_cement_ME_elec_edge[8760] + vFLOW_MA_to_ME_elec_edge[8760] + vFLOW_storage_ME_disc

In [ ]:
i = 1
plant_results = DataFrame(
    elec_edge = Macro.value.(Macro.flow(system.assets[i].elec_edge)).data,
    fuel_edge = Macro.value.(Macro.flow(system.assets[i].fuel_edge)).data,
    cement_edge = Macro.value.(Macro.flow(system.assets[i].cement_edge)).data,
    co2_edge = Macro.value.(Macro.flow(system.assets[i].co2_edge)).data,
)